<a href="https://colab.research.google.com/github/MinahilAzeem/conversational_chatbot/blob/main/Conversational_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
import numpy as np
import random
import string

import bs4 as bs
import urllib.request
import re

In [3]:
get_link = urllib.request.urlopen('https://en.wikipedia.org/wiki/Disease')
get_link = get_link.read()

data = bs.BeautifulSoup(get_link, 'lxml')

data_paragraphs = data.find_all('p')

data_text = ''

for para in data_paragraphs:
    data_text += para.text

data_text = data_text.lower()

In [4]:
data_text = re.sub(r'\[[0-9]*\]', ' ', data_text)
data_text = re.sub(r'\s+', ' ', data_text)

In [6]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [13]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [15]:
data_sentences = nltk.sent_tokenize(data_text)
data_words = nltk.word_tokenize(data_text)

In [16]:
wnlemmatizer = nltk.stem.WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [wnlemmatizer.lemmatize(token) for token in tokens]

punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

In [17]:
greeting_inputs = ("hey", "good morning", "good evening", "morning", "evening", "hi", "whatsup")
greeting_responses = ["hey", "hey hows you?", "*nods*", "hello, how you doing", "hello", "Welcome, I am good and you"]

def generate_greeting_response(greeting):
    for token in greeting.split():
        if token.lower() in greeting_inputs:
            return random.choice(greeting_responses)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
def generate_response(user_input):
    bot_response = ''
    data_sentences.append(user_input)

    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')
    all_word_vectors = word_vectorizer.fit_transform(data_sentences)
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
    similar_sentence_number = similar_vector_values.argsort()[0][-2]

    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        bot_response = bot_response + "I am sorry, I could not understand you"
        return bot_response
    else:
        bot_response = bot_response + data_sentences[similar_sentence_number]
        return bot_response

In [20]:
continue_dialogue = True
print("Hello, I am from AI Medicare. You can ask me any question regarding Diseases:")
while(continue_dialogue == True):
    human_text = input()
    human_text = human_text.lower()
    if human_text != 'bye':
        if human_text == 'thanks' or human_text == 'thank you very much' or human_text == 'thank you':
            continue_dialogue = False
            print("AI Medicare: Most welcome")
        else:
            if generate_greeting_response(human_text) != None:
                print("AI Medicare: " + generate_greeting_response(human_text))
            else:
                print("AI Medicare: ", end="")
                print(generate_response(human_text))
                data_sentences.remove(human_text)
    else:
        continue_dialogue = False
        print("AI Medicare: Good bye and take care of yourself...")

Hello, I am from AI Medicare. You can ask me any question regarding Diseases:
what are infectious diseases?
AI Medicare: infectious diseases can be transmitted, e.g.
Epidemiology
AI Medicare: epidemiology is the study of the factors that cause or encourage diseases.
Epilepsy?
AI Medicare: epilepsy is considered a sign of spiritual gifts among the hmong people.
Airborne disease
AI Medicare: there are four main types of disease: infectious diseases, deficiency diseases, hereditary diseases (including both genetic and non-genetic hereditary diseases), and physiological diseases.
what is pain medicine
AI Medicare: pain management (also called pain medicine) is that branch of medicine employing an interdisciplinary approach to the relief of pain and improvement in the quality of life of those living with pain.
what is social stigma?
AI Medicare: fear of disease can still be a widespread social phenomenon, though not all diseases evoke extreme social stigma.
what do you know about cancer?
AI